#Prep

In [ ]:
import tweepy
import pandas as pd
import time
import re

###Getting user names

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_base = pd.read_csv('/content/drive/MyDrive/funding-dominance/raw-data/funding_raw.csv')
df_usernames = pd.read_csv('/content/drive/MyDrive/funding-dominance/raw-data/twitter_handles.csv')

In [ ]:
users = df_base[['Deal ID', 'Companies', 'CEO (at time of deal)', 'CEO PBId']].copy()
users['username'] = df_usernames['Twitter Handle']

###Cleaning user names

In [ ]:
for i in range(users.shape[0]):
  users['username'][i] = re.sub(r'@', '', str(users['username'][i]))
  users['username'][i] = re.sub(r'\*', '', str(users['username'][i]))
  if '**' in users['username'][i]:
    users['username'][i] = ''
  if users['username'][i] == 'nan':
    users['username'][i] = ''
  if ',' in users['username'][i]: # deleting ambiguous usernames (n=131), unable to verify
    users['username'][i] = ''

In [ ]:
empty = 0
handle = 0
for i in range(users.shape[0]):
  if users['username'][i] == '':
    empty +=1
  else:
    handle +=1
print('rows with twitter handle:', handle, '  ||  rows without twitter handle:', empty, '(exclude)')

rows with twitter handle: 2046   ||  rows without twitter handle: 807 (exclude)


#Scraper

In [ ]:
usernames = users['username'].tolist()
usernames = [x for x in usernames if x]
print(len(usernames), usernames[:20])

2046 ['genioho', 'swehan', 'TousinaMartin', 'NiftyRidley', 'JasonAWood1', 'josh_simons_', 'WilderRumpf', 'UglyAmerican', 'Golfsullivan', 'OliverHeartmont', 'luliu', 'BryanFinnerty', 'kledaras', 'taylormeyer721', 'Garymarella', 'gregg_murset', 'ges13', 'Jesse_Anglen', 'berimeric', 'YaaniYulia']


In [ ]:
consumer_key = "cdV7bjdgWQSxMMVCoAVjUcFtu"
consumer_secret = "m0pUurMEa4vGqlkynbEBVFiOye9tjZSDsVlGadsPnF73iHvClA"
access_token = "1397642727517478918-SGxRcmFwS3zBDDCrPx8Vt4RfE93GOO"
access_token_secret = "FMYoaEGWpNrB6uJN8jhRu3L5GPYvGwWWk4FSvkynsyGpG"

In [ ]:
def username_tweets_to_csv(username,count):
    try:      
        # Creation of query method using parameters
        tweets = tweepy.Cursor(api.user_timeline,screen_name=username, tweet_mode="extended", include_rts=False,).items(count)

        # Pulling information from tweets iterable object
        tweets_list = [[tweet.created_at, tweet.id, tweet.full_text] for tweet in tweets]

        # Creation of dataframe from tweets list
        # Add or remove columns as you remove tweet information
        tweets_df = pd.DataFrame(tweets_list,columns=['Datetime', 'Tweet Id', 'Text'])

        # Converting dataframe to CSV 
        tweets_df.to_csv('/content/drive/MyDrive/funding-dominance/data-collection/tweets/{}-tweets.csv'.format(username), sep=',', index = False)

    except BaseException as e:
          print(username, '- failed on_status,',str(e))
          time.sleep(3)

Batching; otherwise API starts throwing errors

In [ ]:
#adjust batch size:
batch_size = 25

n = len(usernames)
batches = n // batch_size
batch_rest = n % batches

print('batches =', batches, '|| batches_size =', batch_size, '|| batch_rest =', batch_rest)

batches = 81 || batches_size = 25 || batch_rest = 21




---

partial loop starting at breaking point of last iteration

In [ ]:
index = usernames.index('jerondanielpaul')

In [ ]:
breaking = index//batch_size
print('interrupted batch', breaking, 'at index', index, '--> restarting at index', breaking*batch_size)

interrupted batch 60 at index 1502 --> restarting at index 1500


In [ ]:
count = 1000


for batch in range(breaking,batches):
  start = batch * batch_size
  end = start + batch_size - 1

  #Twitter API login; re-entering for every batch since this solved errors during testing
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)
  api = tweepy.API(auth,wait_on_rate_limit=True)

  #calling tweet scraper for every user
  for i in range(start, end):
    username = usernames[i]
    username_tweets_to_csv(username, count)

  print('Batch', batch, '/', batches, 'done.')
  #deleted sleep timer sinze wait_on_rate_limit=True when authenticating. This should do exactly what i was trying to achieve with that.

for i in range(n - batch_rest, n):
  username = usernames[i]
  username_tweets_to_csv
print('Final batch done.')

Batch 60 / 81 done.
Batch 61 / 81 done.
celigonic - failed on_status, Twitter error response: status code = 404
MattElisSD - failed on_status, Twitter error response: status code = 404
Batch 62 / 81 done.
coterienIns - failed on_status, Twitter error response: status code = 404
CompStackCEO - failed on_status, Twitter error response: status code = 404
Batch 63 / 81 done.
sedairus7 - failed on_status, Twitter error response: status code = 404
Batch 64 / 81 done.
Batch 65 / 81 done.
nikil - failed on_status, Twitter error response: status code = 503
Batch 66 / 81 done.
Batch 67 / 81 done.
Andrelcollins1 - failed on_status, Twitter error response: status code = 404
naftaharris - failed on_status, Twitter error response: status code = 404
Batch 68 / 81 done.
bforgan - failed on_status, Twitter error response: status code = 401
wgapatrick - failed on_status, Twitter error response: status code = 404
mrsrobinsin_a - failed on_status, Twitter error response: status code = 404
ajrathner - fail

Scraping Loop

In [ ]:
#specify number of tweets to scrape per user
count = 1000


for batch in range(batches):
  start = batch * batch_size
  end = start + batch_size - 1

  #Twitter API login; re-entering for every batch since this solved errors during testing
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)
  api = tweepy.API(auth,wait_on_rate_limit=True)

  #calling tweet scraper for every user
  for i in range(start, end):
    username = usernames[i]
    username_tweets_to_csv(username, count)

  print('Batch', batch, '/', batches, 'done.')
  time.sleep(5*60)

# remaining batch
for i in range(int(len(usernames)-batch_rest), len(usernames)):
  username = usernames[i]
  username_tweets_to_csv